## GStreamer Basics - basic pipeline read buffer

### Resources:

Code adapted from:
https://paulbridger.com/posts/video-analytics-pytorch-pipeline/


Gstreamer References:
https://developer.download.nvidia.cn/embedded/L4T/r32_Release_v1.0/Docs/Accelerated_GStreamer_User_Guide.pdf

DeepStream:
https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvinfer.html

In [ ]:
import os, sys
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst

In [2]:
# Simple function to read buffer

def on_frame_probe_simple(pad, info):
    buffer = info.get_buffer()
    print(f'[{buffer.pts / Gst.SECOND:6.2f}]')
    return Gst.PadProbeReturn.OK

In [14]:
## Some demo Gstreamer pipelines - change the number of buffers if you want to limit it. 

# Demo  video
v1 = """
filesrc location=in.mp4 num-buffers=200 !
decodebin !
fakesink name=s
"""

# Basic Video stream
v2 = """
v4l2src num-buffers=100 device=/dev/video0 ! 
xvimagesink name= webcam_stream 
"""

# Aswins stream - does not work with my camera - dimensions are not supported. 
v3 = """
v4l2src num-buffers=1000 device=/dev/video0 ! 
nvvidconv ! 
video/x-raw(memory:NVMM),width=404,height=224,framerate=30/1 ! 
nvvidconv top=0 bottom=224 left=90 right=314 ! 
video/x-raw(memory:NVMM),width=224,height=224,framerate=30/1 ! 
omxh265enc insert-vui=1 ! 
h265parse ! 
rtph265pay config-interval=1 !
fakesink sync=false name= webcam_stream
"""

# Pytorch Demo Video from file
v4 = """
filesrc location=../notebooks/source_video.mp4 num-buffers=100 !
decodebin !
nvvidconv !
video/x-raw,format=RGBA !
fakesink name=webcam_stream
"""

# Stream for my device - no display on screen
v5 = """
v4l2src device=/dev/video0 ! nvvidconv ! \
video/x-raw(memory:NVMM),framerate=(fraction)30/1,width=320,height=240 ! \
nvvidconv top=0 bottom=240 left = 90 right=320 ! \
video/x-raw(memory:NVMM),format=RGBA,width=224,height=224 ! \
fakesink name=webcam_stream
"""

# From command line
# gst-launch-1.0 v4l2src device=/dev/video0 ! nvvidconv ! \
# 'video/x-raw(memory:NVMM),framerate=(fraction)30/1,width=320,height=240' ! \
# nvvidconv top=0 bottom=240 left = 90 right=320 ! \
# 'video/x-raw(memory:NVMM),format=RGBA,width=224,height=224' ! \
# nvvidconv ! nvegltransform ! nveglglessink -e

# Stream with a seperate tee for displaying on the screen. 
v6 = """
v4l2src device=/dev/video0 ! nvvidconv ! \
video/x-raw(memory:NVMM),framerate=(fraction)30/1,width=320,height=240 ! \
nvvidconv top=0 bottom=240 left = 90 right=320 ! \
video/x-raw(memory:NVMM),format=RGBA,width=224,height=224 ! \
queue ! tee name=t t. ! queue ! fakesink name=webcam_stream sync=false t. ! \
queue ! nvvidconv ! nvegltransform ! nveglglessink
"""
# From command line
# gst-launch-1.0 v4l2src device=/dev/video0 ! nvvidconv ! \
# 'video/x-raw(memory:NVMM),framerate=(fraction)30/1,width=320,height=240' ! \
# nvvidconv top=0 bottom=240 left = 90 right=320 ! \
# 'video/x-raw(memory:NVMM),format=RGBA,width=224,height=224' ! \
# queue ! tee name=t t. ! queue ! fakesink name=webcam_stream sync=false t. ! \
# queue ! nvvidconv ! nvegltransform ! nveglglessink -e

#Selector Variable
stream_string = v4

In [15]:
# Instantiate Pipeline
Gst.init()

# Launch Pipeline
pipeline = Gst.parse_launch(f"{stream_string}")

# Probe read buffer and apply callback 'on_frame_probe'
pipeline.get_by_name('webcam_stream').get_static_pad('sink').add_probe(
    Gst.PadProbeType.BUFFER,
    on_frame_probe_simple
)

# Set Pipeline to PLAYING
pipeline.set_state(Gst.State.PLAYING)

# Scan the pipeline bus for errors and other messages.
try:
    while True:
        msg = pipeline.get_bus().timed_pop_filtered(
            Gst.SECOND,
            Gst.MessageType.EOS | Gst.MessageType.ERROR
        )
        if msg:
            text = msg.get_structure().to_string() if msg.get_structure() else ''
            msg_type = Gst.message_type_get_name(msg.type)
            print(f'{msg.src.name}: [{msg_type}] {text}')
            break
            
# Break
except KeyboardInterrupt:
    print ('KeyboardInterrupt exception is caught')
    
# Write pipeline Graph and close pipeline
finally:
    print('Writing graph...')
    open(f'logs/logs.txt', 'w', encoding="utf8").write(
        Gst.debug_bin_to_dot_data(
            pipeline, Gst.DebugGraphDetails.ALL
        )
    )
    pipeline.set_state(Gst.State.NULL)
print('done.')

[  0.00]
[  0.05]
[  0.10]
[  0.15]
[  0.20]
[  0.25]
[  0.30]
[  0.35]
[  0.40]
[  0.45]
[  0.50]
[  0.55]
[  0.60]
[  0.65]
[  0.70]
[  0.75]
[  0.80]
[  0.85]
[  0.90]
[  0.95]
[  1.00]
[  1.05]
[  1.10]
[  1.15]
[  1.20]
[  1.25]
[  1.30]
[  1.35]
[  1.40]
[  1.45]
[  1.50]
[  1.55]
[  1.60]
[  1.65]
[  1.70]
[  1.75]
[  1.80]
[  1.85]
[  1.90]
[  1.95]
[  2.00]
[  2.05]
[  2.10]
[  2.15]
[  2.20]
[  2.25]
[  2.30]
[  2.35]
[  2.40]
[  2.45]
[  2.50]
[  2.55]
[  2.60]
[  2.65]
[  2.70]
[  2.75]
[  2.80]
[  2.85]
[  2.90]
[  2.95]
[  3.00]
[  3.05]
[  3.10]
[  3.15]
[  3.20]
[  3.25]
[  3.30]
[  3.35]
[  3.40]
[  3.45]
[  3.50]
[  3.55]
[  3.60]
[  3.65]
[  3.70]
[  3.75]
[  3.80]
[  3.85]
[  3.90]
[  3.95]
[  4.00]
[  4.05]
[  4.10]
[  4.15]
[  4.20]
[  4.25]
[  4.30]
[  4.35]
[  4.40]
[  4.45]
[  4.50]
[  4.55]
[  4.60]
pipeline5: [eos] 
Writing graph...
done.


In [16]:
pipeline.set_state(Gst.State.NULL)

<enum GST_STATE_CHANGE_SUCCESS of type Gst.StateChangeReturn>